In [ ]:
# Nota: si necesitas instalar dependencias localmente, ejecuta en terminal:
# py -3.11 -m pip install -r requirements.txt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import json

In [ ]:
# Constantes y carga de datos
DATA_PATH = 'data/raw/Mushrooms-Dataset.csv'
CARACTERISTICAS_SELECCIONADAS = [
    'bruises', 'gill-color', 'gill-size', 'gill-spacing', 'habitat',
    'odor', 'population', 'ring-type', 'spore-print-color',
    'stalk-color-above-ring', 'stalk-color-below-ring', 'stalk-root',
    'stalk-surface-above-ring', 'stalk-surface-below-ring'
]

df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
df.head()

In [ ]:
# EDA básico
print('Valores faltantes totales:', df.isnull().sum().sum())
print('
Distribución de la variable target (class):')
print(df['class'].value_counts())

print('
Valores únicos por columna:')
print(df.nunique())

In [ ]:
# Correlación (tras encoding) para analizar relación con la clase
encoded = pd.get_dummies(df, drop_first=False)
# identificar columna target (comienza con 'class')
target_col = [c for c in encoded.columns if c.startswith('class')][0]
cor = encoded.corr()
cor_target = cor[target_col].drop(target_col)
low_corr = cor_target[abs(cor_target) < 0.25]
high_corr = cor_target[abs(cor_target) >= 0.25]

unique_low_headers = sorted(list({c.rsplit('_',1)[0] for c in low_corr.index}))
unique_high_headers = sorted(list({c.rsplit('_',1)[0] for c in high_corr.index}))

print('Encabezados con baja correlación (|corr| < 0.25) -- únicos:')
print(unique_low_headers)
print('
Encabezados con alta correlación (|corr| >= 0.25) -- únicos:')
print(unique_high_headers)

In [ ]:
# Preprocesado para entrenamiento (usar solo las características seleccionadas)
X = df[CARACTERISTICAS_SELECCIONADAS].copy()
y = df['class'].map({'e': 0, 'p': 1})
X_encoded = pd.get_dummies(X, drop_first=False)
print('X_encoded shape:', X_encoded.shape)
# Guardar las columnas finales para alinear entradas futuras
FEATURE_COLUMNS = list(X_encoded.columns)
print('Número de features después de encoding:', len(FEATURE_COLUMNS))

In [ ]:
# Entrenamiento del modelo
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.4, stratify=y, random_state=123)
model = LogisticRegression(max_iter=1000, random_state=456, C=0.01)
model.fit(X_train, y_train)

# Predicciones y métricas
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
metrics = {
    'accuracy': float(accuracy_score(y_test, y_pred)),
    'precision': float(precision_score(y_test, y_pred)),
    'recall': float(recall_score(y_test, y_pred)),
    'f1': float(f1_score(y_test, y_pred)),
    'roc_auc': float(roc_auc_score(y_test, y_proba))
}
print('Métricas:')
print(metrics)

# Guardar modelo y métricas
import os
os.makedirs('models', exist_ok=True)
os.makedirs('reports', exist_ok=True)
joblib.dump(model, 'models/modelo_hongos.joblib')
with open('reports/metricas.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print('Modelo y métricas guardados en models/ y reports/')

In [ ]:
# Prueba rápida: construir ejemplo automático y predecir
# Elegimos el primer valor único de cada columna como ejemplo valido
example = {}
for col in CARACTERISTICAS_SELECCIONADAS:
    example[col] = df[col].dropna().unique()[0]

X_ex = pd.DataFrame([example])
X_ex_enc = pd.get_dummies(X_ex, drop_first=False)
# Alinear columnas con FEATURE_COLUMNS
X_final = pd.DataFrame(0, index=[0], columns=FEATURE_COLUMNS)
for c in X_ex_enc.columns:
    if c in X_final.columns:
        X_final[c] = X_ex_enc[c].values[0]

pred = model.predict(X_final)[0]
proba = model.predict_proba(X_final)[0]
print('Ejemplo de entrada:', example)
print('Predicción (0=comestible,1=venenoso):', pred)
print('Probabilidades:', proba)

## Ejecutar localmente
1. Instala dependencias: `py -3.11 -m pip install -r requirements.txt`
2. Inicia Jupyter: `py -3.11 -m notebook` y abre `notebooks/train_mushrooms.ipynb`.

Nota: las rutas son relativas al root del proyecto. El notebook guarda el modelo en `models/modelo_hongos.joblib` y las métricas en `reports/metricas.json`.